<a href="https://colab.research.google.com/github/ZikryRamadhan/Bangkit2021_Pafin/blob/main/Capstone_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

## Getting the dataset

Dataset available from kaggle (https://www.kaggle.com/charanpuvvala/company-classification)

Upload kaggle.json first (open kaggle, account -> create new API token)

In [2]:
! chmod 600 kaggle.json && (ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle) && mv kaggle.json ~/.kaggle/ && echo 'Done'

Done


In [3]:
! kaggle datasets download charanpuvvala/company-classification
! ls

 99% 121M/122M [00:00<00:00, 118MB/s]
100% 122M/122M [00:00<00:00, 132MB/s]
company-classification.zip  sample_data


## Preprocessing dataset

In [2]:
df = pd.read_csv('company-classification.zip')
df.head()

,Category,website,company_name,homepage_text,h1,h2,h3,nav_link_text,meta_keywords,meta_description
0,Commercial Services & Supplies,bipelectric.com,bip dipietro electric inc,Electrici...,NaN,NaN,NaN,NaN,"electricians vero beach, vero beach electrical...","Providing quality, reliable full service resid..."
1,Healthcare,eliasmedical.com,elias medical,site map | en español Elias Medical h...,Offering Bakersfield family medical care from ...,Welcome to ELIAS MEDICAL#sep#Family Medical Pr...,Get To Know Elias Medical#sep#Family Medical P...,NaN,Elias Medical bakersfield ca family doctor med...,For the best value in Bakersfield skin care tr...
2,Commercial Services & Supplies,koopsoverheaddoors.com,koops overhead doors,Home About Us Garage Door Repair & Servi...,NaN,Customer Reviews#sep#Welcome to Koops Overhead...,NaN,NaN,"Koops Overhead Doors, Albany Garage Doors, Tro...","Koops Overhead Doors specializes in the sales,..."
3,Healthcare,midtowneyes.com,midtown eyecare,918-599-0202 Type Size...,NaN,Welcome to our practice!,NaN,NaN,NaN,We would like to welcome you to Midtown Eyecar...
4,Commercial Services & Supplies,reprosecurity.co.uk,repro security ltd,Simply fill out our form below...,NaN,Welcome to REPRO SECURITY Ltd,NaN,NaN,NaN,Repro Security provide a range of tailor made ...


In [3]:
# Remove all column except Category and meta_description 
# Rename meta_description to Description
df.drop(['company_name', 'homepage_text',	'h1',	'h2',	'h3', 'website', 'nav_link_text',	'meta_keywords'], axis=1, inplace=True)
df.rename(columns={'meta_description' : 'Description'}, inplace=True)
df.head()

,Category,Description
0,Commercial Services & Supplies,"Providing quality, reliable full service resid..."
1,Healthcare,For the best value in Bakersfield skin care tr...
2,Commercial Services & Supplies,"Koops Overhead Doors specializes in the sales,..."
3,Healthcare,We would like to welcome you to Midtown Eyecar...
4,Commercial Services & Supplies,Repro Security provide a range of tailor made ...


In [4]:
# Category with more than 1 word will raise an error during training
# So have to rename or remove them
err = ['Commercial Services & Supplies',
       'Energy & Utilities', 'Professional Services',
       'Corporate Services', 'Media, Marketing & Sales',
       'Information Technology', 'Consumer Discretionary', 
       'Transportation & Logistics', 'Consumer Staples']

In [5]:
# We will remove them to make the dataset smaller
for x in range(len(err)):
  df.drop(index=df[df['Category'] == err[x]].index, inplace=True)

In [6]:
# Drop any row with NaN value
df.dropna(inplace=True)

In [7]:
# Shuffle and Reset index
df = df.sample(frac=1).reset_index(drop=True)

In [8]:
df.head()

,Category,Description
0,Healthcare,Visit Indian Walk Veterinary Center in Newtown...
1,Financials,InterWeb Insurance provides our customers acce...
2,Healthcare,Harland Medical Systems is a leading provider ...
3,Financials,"UNITEL is a division of UNICO group, dedicated..."
4,Financials,We are a team of professional mortgage brokers...


## Build the model

reference : https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Exercise%20-%20Question.ipynb

(from Coursera Natural Language Processing in TensorFlow course)

In [9]:
# parameter setting
vocab_size = 1000
embedding_dim = 6 # changing this doesn't seem to make much impact on the model result
max_length = 50 # changing this doesn't seem to make much impact on the model result
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_portion = .8

In [10]:
labels = [x for x in df['Category']]
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [11]:
descriptions = [x for x in df['Description']]
for x in range(len(descriptions)):
  for word in stopwords:
    token = " " + word + " "
    descriptions[x] = descriptions[x].replace(token, " ") 

# check the length to make sure both have same length
print(len(labels))
print(len(descriptions))

18303
18303


In [12]:
train_size = int(len(descriptions) * training_portion)

train_descriptions = descriptions[:train_size]
train_labels = labels[:train_size]

validation_descriptions = descriptions[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_descriptions))
print(len(train_labels))
print(len(validation_descriptions))
print(len(validation_labels))

14642
14642
14642
3661
3661


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_descriptions)
word_index = tokenizer.word_index

train_descriptions = tokenizer.texts_to_sequences(train_descriptions)
train_padded = pad_sequences(train_descriptions, padding=padding_type, maxlen=max_length)

print(len(train_descriptions[0]))
print(len(train_padded[0]))

print(len(train_descriptions[1]))
print(len(train_padded[1]))

print(len(train_descriptions[10]))
print(len(train_padded[10]))

23
50
43
50
30
50


In [14]:
validation_descriptions = tokenizer.texts_to_sequences(validation_descriptions)
validation_padded = pad_sequences(validation_descriptions, padding=padding_type, maxlen=max_length)

print(len(validation_descriptions))
print(validation_padded.shape)

3661
(3661, 50)


In [15]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

print(validation_label_seq[0])
print(validation_label_seq[1])
print(validation_label_seq[2])
print(validation_label_seq.shape)

[1]
[2]
[1]
(14642, 1)
[3]
[1]
[3]
(3661, 1)


In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    # tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 6)             6000      
_________________________________________________________________
global_average_pooling1d (Gl (None, 6)                 0         
_________________________________________________________________
dense (Dense)                (None, 16)                112       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 85        
Total params: 6,197
Trainable params: 6,197
Non-trainable params: 0
_________________________________________________________________


In [17]:
num_epochs = 15
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq))

Epoch 1/15
458/458 [==============================] - 1s 2ms/step - loss: 1.3427 - accuracy: 0.4748 - val_loss: 1.1722 - val_accuracy: 0.6528
Epoch 2/15
458/458 [==============================] - 1s 2ms/step - loss: 0.9189 - accuracy: 0.6918 - val_loss: 0.6674 - val_accuracy: 0.7703
Epoch 3/15
458/458 [==============================] - 1s 2ms/step - loss: 0.5350 - accuracy: 0.8030 - val_loss: 0.4676 - val_accuracy: 0.8014
Epoch 4/15
458/458 [==============================] - 1s 2ms/step - loss: 0.4133 - accuracy: 0.8224 - val_loss: 0.4006 - val_accuracy: 0.8388
Epoch 5/15
458/458 [==============================] - 1s 2ms/step - loss: 0.3578 - accuracy: 0.8546 - val_loss: 0.3583 - val_accuracy: 0.8580
Epoch 6/15
458/458 [==============================] - 1s 2ms/step - loss: 0.3096 - accuracy: 0.8973 - val_loss: 0.3170 - val_accuracy: 0.8946
Epoch 7/15
458/458 [==============================] - 1s 2ms/step - loss: 0.2664 - accuracy: 0.9188 - val_loss: 0.2901 - val_accuracy: 0.9017
Epoch 

## Test Predict

In [18]:
# check word token of the category
label_tokenizer.index_word

{1: 'healthcare', 2: 'financials', 3: 'industrials', 4: 'materials'}

In [24]:
text = "creating herbal medicine recommended by various doctor" 
test_string = [text]
test = tokenizer.texts_to_sequences(test_string)
test_padded = pad_sequences(test, padding=padding_type, maxlen=max_length)

In [ ]:
test_padded

In [25]:
res = model.predict(test_padded)
result = np.argmax(res, axis=1)
result

array([1])

## Save the model

In [22]:
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [23]:
# Make zipfile from the saved model to deploy to cloud
import shutil
shutil.make_archive('saved_model', 'zip', 'saved_model')

'/content/saved_model.zip'